In [72]:
import numpy as np
import pandas as pd
import datetime
import math

In [73]:
dam = pd.read_csv('dam.csv')
dam

,market,date,hour,price,volume
0,1,01/01/2016,0,40.10,477.5
1,1,01/01/2016,1,40.08,486.6
2,1,01/01/2016,2,40.06,495.6
3,1,01/01/2016,3,40.08,496.6
4,1,01/01/2016,4,40.07,502.9
...,...,...,...,...,...
44587,1,29/01/2021,13,61.97,134.0
44588,1,29/01/2021,23,58.05,59.0
44589,1,30/01/2021,17,42.08,64.0
44590,1,31/01/2021,3,42.19,149.0


In [75]:
dam['date']=pd.to_datetime(dam['date'],format="%d/%m/%Y")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam

,market,date,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,,
2016-01-01,1,2016-01-01,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0
2016-01-01,1,2016-01-01,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1
2016-01-01,1,2016-01-01,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2
2016-01-01,1,2016-01-01,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3
2016-01-01,1,2016-01-01,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-29,1,2021-01-29,13,61.97,134.0,2021,1,29,4,2021.01,weekday_13
2021-01-29,1,2021-01-29,23,58.05,59.0,2021,1,29,4,2021.01,weekday_23
2021-01-30,1,2021-01-30,17,42.08,64.0,2021,1,30,5,2021.01,weekend_17


In [76]:
monthly=dam.groupby(['month_no','year','month','hour', 'hour_ref']).mean().reset_index(['hour','month','year'])
monthly=monthly.drop(['day'],1)
monthly.rename(columns={'price': 'price_mean'}, inplace=True)
monthly['volume_sum']=dam[['month_no','hour_ref','volume']].groupby(['month_no','hour_ref']).sum()
monthly['hours']=dam[['month_no','hour_ref']].groupby(['month_no','hour_ref']).size()
monthly['power']=monthly['volume_sum']/monthly['hours']
monthly

year  month  hour  market  price_mean      volume  \
month_no hour_ref                                                        
2016.01  weekday_0   2016      1     0     1.0   48.505238  332.371429   
         weekend_0   2016      1     0     1.0   48.618000  372.530000   
         weekday_1   2016      1     1     1.0   48.434286  336.747619   
         weekend_1   2016      1     1     1.0   48.459000  372.900000   
         weekday_2   2016      1     2     1.0   48.385714  339.400000   
...                   ...    ...   ...     ...         ...         ...   
2021.01  weekend_21  2021      1    21     1.0   49.518000   84.900000   
         weekday_22  2021      1    22     1.0   55.550952  109.528571   
         weekend_22  2021      1    22     1.0   52.735000  115.700000   
         weekday_23  2021      1    23     1.0   50.306667  122.947619   
         weekend_23  2021      1    23     1.0   47.540000  119.840000   

                     day_of_week  volume_sum  hours       power  
month_no hour_ref                                                
2016.01  weekday_0      2.095238      6979.8     21  332.371429  
         weekend_0      5.500000      3725.3     10  372.530000  
         weekday_1      2.095238      7071.7     21  336.747619  
         weekend_1      5.500000      3729.0     10  372.900000  
         weekday_2      2.095238      7127.4     21  339.400000  
...                          ...         ...    ...         ...  
2021.01  weekend_21     5.500000       849.0     10   84.900000  
         weekday_22     2.095238      2300.1     21  109.528571  
         weekend_22     5.500000      1157.0     10  115.700000  
         weekday_23     2.095238      2581.9     21  122.947619  
         weekend_23     5.500000      1198.4     10  119.840000  

[2928 rows x 10 columns]

In [101]:
dam_filt=dam[(dam.year<2020)]
cal_monthly=dam_filt[['month','hour_ref','price']].groupby(['month','hour_ref']).mean()

cal_monthly.rename(columns={'price': 'price_mean'}, inplace=True)
cal_monthly['hours']=dam_filt[['month','hour_ref']].groupby(['month', 'hour_ref']).size()
cal_monthly['volume_sum']=dam_filt[['month','hour_ref','volume']].groupby(['month','hour_ref']).sum()
cal_monthly['power']=cal_monthly['volume_sum']/cal_monthly['hours']
cal_monthly

price_mean  hours  volume_sum       power
month hour_ref                                             
1     weekday_0    33.593483     89     23417.5  263.117978
      weekday_1    33.247303     89     23332.7  262.165169
      weekday_10   74.164831     89      7730.5   86.859551
      weekday_11   72.023371     89      8090.3   90.902247
      weekday_12   71.558539     89      8277.5   93.005618
...                      ...    ...         ...         ...
12    weekend_5    32.883158     38      9776.0  257.263158
      weekend_6    34.064737     38      9188.3  241.797368
      weekend_7    46.835526     38      8213.6  216.147368
      weekend_8    52.244211     38      7889.7  207.623684
      weekend_9    53.827105     38      7861.4  206.878947

[576 rows x 4 columns]

In [15]:
monthly.to_csv('dam_monthly_hours.csv')

In [102]:
cal_monthly.to_csv('dam_cal_monthly_hours.csv')

In [52]:
dam.to_csv('dam_out.csv')